In [2]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("..")
from utils import *
import warnings
warnings.filterwarnings("ignore")
import os
import psutil

pd.options.display.max_columns=1000
pd.options.display.max_rows=1000

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from catboost import CatBoostClassifier, Pool

In [ ]:
def Kfold(model, k, X_train, Y_train, test, is_test = False):

    valid_arrays = []
    test_arrays = []
    soft_voting_value = np.zeros((len(test)))

    S_kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    X_train.reset_index(drop = True, inplace=True)
    Y_train.reset_index(drop = True, inplace=True)

    for iter, (train_index, test_index) in enumerate(S_kfold.split(X_train, Y_train)):
        x_train, x_test = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train, y_test = Y_train.iloc[train_index], Y_train.iloc[test_index]
        model.fit(x_train, y_train, eval_metric='AUC')
        valid_pred = model.predict_proba(x_test)
        valid_arrays.append(valid_pred[:, 1])
        score = roc_auc_score(y_test, valid_pred[:, 1])
        print(f'---------------- {iter+1} fold의 Acc: {score} ----------------')

        if is_test:
            ''' soft voting '''
            test_pred = model.predict_proba(test)
            test_arrays.append(test_pred[:, 1])
            for value in test_arrays:
                print(soft_voting_value)
                soft_voting_value += value
                print(soft_voting_value)
    soft_voting_value /= k

    return valid_arrays, test_arrays, soft_voting_value

In [3]:
''' Seed '''
seed_everything(42)

''' Data Load '''
train = pd.read_csv("/home/workspace/DACON/Click_predict/data/train3.csv")
test = pd.read_csv("/home/workspace/DACON/Click_predict/data/test3.csv")
# train = preprocessing(train)
# test = preprocessing(test, is_test = True)

''' Train / Vaild Split'''
param = {'max_depth': 30,
 'num_leaves': 306,
 'subsample': 0.9757365622458185,
 'subsample_freq': 8,
 "n_estimators" : 1000,
 'min_child_samples': 136}

X_train = train.drop(columns = ["Click"]) 
y_train = train["Click"]

''' Model '''
model = lgb.LGBMClassifier(random_state=42, **param)

''' K-Fold '''
valid_array, test_array, voting = Kfold(model, 7, X_train, y_train, test, True)

: 